# **Este *ipynb* está enmarcado dentro de un trabajo que busca predecir qué alumnos del departamento de Sistemas la UTN FRBA desertarán.**

Los datos disponibles se presentan en tres archivos separados.

Los registros de estas tres tablas se encuentran relacionados a través de un ID anonimizado que corresponde a un estudiante en particular. 

En este **ipynb** se preprocesan los datos disponibles hasta llegar a unificarlos en una sola tabla, la cual denominaremos **merged_df**.


## **Aclaraciones**

Aquellas líneas de código identificadas con **" # "** fueron extraídas de la siguente fuente:
* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/blob/master/desercion/munge/00_datos_01/Giar%2020-09.ipynb

Aquellas líneas de código identificadas con **" ## "** son de elaboración propia.

## **Definiciones**

* **Actividad​**: Se considera que un estudiante tiene ​actividad en un año determinado si: tiene alguna materia con el campo “Tipo de aprobación” distinto de “libre” ó si rindió algún final de esa u otra materia. 
 *Notar que ​actividad es una característica siempre asociada a un año determinado ​XX​.*
---
* (ALF) **Año Lectivo Final**​: Año que se toma como punto de partida para analizar HACIA ATRÁS las historias disponibles hasta 2008. En nuestro caso, por ahora, **ALF = 2016**. 
---
* (ALI) **Año Lectivo Inicial**: Año a partir de que contamos con datos. En nuestro caso, por ahora; **ALI = 2008**. 
---
* (AUA) **Año de Última Actividad​**: Último año en que el estudiante haya tenido alguna actividad​. 
---
* (AI) **Año de Ingreso**​: Año en el que el estudiante haya realizado su primera actividad.
---
* **Alumno ​activo a la fecha ​XX**​: Se considera que un estudiante está “activo a la fecha ​XX​” si en el año ​XX ​registró ​actividad​.  
---
* (AE) **Año de egreso**​: Año en que aprobó “Proyecto Final”.  
---
* **Condición de Egresado​**: La condición de ​egresado se puede asignar a un alumno en el año de egreso o posteriores. *Ver que se necesita indicar un año para ver si pertenece a esa categoría.* 
---
* **Condición de Desertor**​: Se considera que un alumno es desertor para el año ​XX si **no** registra actividad ni en el año ​XX ni en el año ​XX - 1​. *Ver que se necesita indicar un año para ver si pertenece a esa categoría.*
---

## **Importación de librerías**





In [ ]:
## Importamos librerías para manipulación de datos.
import pandas as pd
import numpy as np

In [ ]:
## Importo librerías para calcular distancias entre ciudadades 
  
  ## Elemento diferencial respecto al análisis realizado por el GIAR en 2020 ##

from geopy.geocoders import GoogleV3
from geopy import distance

## **Defino años para el análisis**

In [ ]:
ALF = 2016 ## Año lectivo final
ALI = 2008 ## Año lectivo inicial

#Seteo el ano sobre el que voy a medir
ano = 2017

## **Google Colaboratory o Local**
El notebook podrá ser corrido tanto localmente como en Google Colaboratory.

El usuario deberá modificar el root path de acuerdo a su conveniencia.


In [ ]:
## Verificamos si estamos corriendo el noteboock en Google Colaboratory.
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)

## En el caso de estar en Google Colab, montamos nuestro Drive.
if var_google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  ## Direccion root donde está el notebook.
  root_path = "/content/gdrive/MyDrive/Colab Notebooks/GIAR/"

## En el caso de no estar en Google Colab, estamos corriendo localmente el notebook.
else:
  root_path = ""

True
Mounted at /content/gdrive


## **Datasets**

In [ ]:
## Importamos el dataset Datos-Alumnos-SIGA.
alumnos = pd.read_csv(root_path + 'datos/alumnos_desertores.csv')

In [ ]:
## Importamos el dataset Cursadas-Alumnos-SIGA.
cursadas = pd.read_csv(root_path + 'datos/Cursadas-Alumnos-SIGA.csv')

In [ ]:
## Importamos el dataset Finales-Alumnos-SIGA.
finales = pd.read_csv(root_path + 'datos/Finales-Alumnos-SIGA.csv')

## **PREPROSESAMIENTO DE LOS DATOS**

### **ALUMNOS** ###

#### Variables que se decidieron sacar:
- **Pais**
- **Estado Civil**


#### Variables que se van a generar:
- **EsTecnico** como una categorica que es "SI" en el caso de que sea tecnico y "NO" en otro caso. Se deben respetar los nulos.
- **Distancia** como una numérica que medirá la distancia en KM que existe entre el domicilio declarado del alumno y la universidad. Dado que la FRBA cuenta con dos sedes, se fijará como locación de la misma el punto medio entre ambas sedes. En este caso, Av. Directorio 1150, CABA.
En caso que la distancia sea mayor a 70 KM, se supondrá que el dato del domicilio está desatualizado y se desestimará dicho registro. Se deben respetar los nulos.

In [ ]:
## Eliminamos aquellos registros en los que no se cuenta con información.  
## alumnos.dropna(subset = ['Estudios Secundarios'], how ='any', inplace = True)

In [ ]:
#Genero la nueva columna de esTecnico
alumnos['EsTecnico']=alumnos.apply(lambda row: 1 if row['Estudios Secundarios']=='Técnico'  else (row['Estudios Secundarios'] if pd.isnull(row['Estudios Secundarios']) else 0), axis=1)

In [ ]:
#Genero el dataset sin las variables que se decidieron
alumnos_df = alumnos[['Codigo Alumno','EsTecnico','deserto']]

### **CURSADAS** ###

#### Variables que se decidieron sacar:
- Curso
- Materia
- Departamento
- Ciclo Lectivo (la vamos a sacar pero sera utilizada para filtrar los registros)
- Modalidad


#### Variables que se van a generar:
- Edad al ingreso (Ano ingreso - Ano nacimiento) 
- Turno una columna por turno con la cantidad 
- Tipo de aprobación por cada una de las variables la cantidad 
- Maximo recursada por materias (existe mas de un registro por alumno y materia)
- Cantidad de veces recursada regular OK
- Descripción de recursada regular OK
- Edad en ano de ultima actividad

Tener en cuenta que se debe guardar el ciclo lectivo mas actual

In [ ]:
#Filtro el set de datos por el ano en que lo quiero medir
cursadas=cursadas.loc[cursadas['Ciclo Lectivo de Cursada']<ano-2,:]

In [ ]:
# Genero la variable edad al ingreso
cursadas['edad al ingreso']=cursadas['Año de ingreso']-cursadas['Año de nacimiento']

#Genero la lista de las descripciones de recursadas
recursadas_df=cursadas.groupby(['Cantidad de veces recursada regular','Descripción de recursada regular'])['Materia'].max().reset_index()[['Cantidad de veces recursada regular','Descripción de recursada regular']]

#Agrupo las cursadas de cada alumno por materia y calculo la mayor cantidad de recursadas y lo uno con la descripcion de cada una
cursadas_df=cursadas.groupby(['Codigo Alumno','Materia'])['Cantidad de veces recursada regular'].max().reset_index().join(recursadas_df.set_index('Cantidad de veces recursada regular'),on='Cantidad de veces recursada regular',rsuffix='_o')

#Compruebo con un alumno si se obtuvieron los registros correctos
print(cursadas_df[cursadas_df['Codigo Alumno']==9996270])

#Setea la descripcion como categoria
cursadas_df['Descripción de recursada regular']=cursadas_df['Descripción de recursada regular'].astype('category')

# Elimino la columna materia, y agrupo por alumno y sumarizo las descripciones
recursadas_df=pd.get_dummies(cursadas_df,columns=['Descripción de recursada regular']).drop(['Materia'], axis=1).groupby(['Codigo Alumno']).sum()

       Codigo Alumno                           Materia  \
70723        9996270  Algoritmos y Estructura de Datos   
70724        9996270             Análisis Matemático I   
70725        9996270      Arquitectura de Computadores   
70726        9996270               Matemática Discreta   
70727        9996270                           Química   
70728        9996270         Sistemas y Organizaciones   
70729        9996270     Álgebra y Geometría Analítica   

       Cantidad de veces recursada regular Descripción de recursada regular  
70723                                    1                    Recurso 1 Vez  
70724                                    1                    Recurso 1 Vez  
70725                                    0                       No Recurso  
70726                                    1                    Recurso 1 Vez  
70727                                    0                       No Recurso  
70728                                    0                       No

In [ ]:
#Genero las dummies que se acordaron y los valores restantes
cursadas_df=pd.get_dummies(cursadas,columns=['Turno','Tipo de aprobación']).groupby(['Codigo Alumno']).agg({'Ciclo Lectivo de Cursada':['max'],
        'edad al ingreso':['max'],'Año de ingreso':['min'], 'Turno_Mañana':['sum'],
       'Turno_Noche':['sum'], 'Turno_Tarde':['sum'], 'Tipo de aprobación_Cambio Curso':['sum'],
        'Tipo de aprobación_Firmo':['sum'],
       'Tipo de aprobación_Libre':['sum'], 'Tipo de aprobación_No Firmo':['sum'],
       'Tipo de aprobación_Promociono':['sum']
        ,'Sexo':['max']})

#Corrijo los nombres de las columnas
cursadas_df.columns = [col[0] for col in cursadas_df.columns.values]

#Uno este DataFrame con el calculado con las descripciones de recursadas y lo guardo en el dataframe de cursadas
cursadas_df=cursadas_df.reset_index().join(recursadas_df,on='Codigo Alumno')

### **FINALES** ###

#### Columnas a sacar ####
- Materia
- Ano (Solo se usa para filtrar)


#### Variables a generar
- Promedio sobre los maximos de Nota
- Promedio sobre la nota (con aplazos)
- Cantidad de veces que aprobo
- Cantidad de veces que no aprobo
- Cantidad de veces que promociono

In [ ]:
#Filtro el set de datos por el ano en que lo quiero medir
finales=finales.loc[finales['Año']<ano-2,:]

In [ ]:
##
finales['Nota'].value_counts()

2     9941
7     9601
4     7676
8     7591
6     5860
5     4723
9     4547
10    3929
1      476
3       47
0       14
Name: Nota, dtype: int64

In [ ]:
#Corrijo los finales para que no haya notas con 11
finales.loc[finales['Nota']==11,['Nota']]=10
#Genero la columna de no aprobado
finales['noAprobado']=finales['Aprobado'].apply(lambda row: 1 if row==0 else 0)
#Agrupo los finales y me quedo con el promedio de las notas maximas
finales_max_df=finales.groupby(['Codigo Alumno','Materia'])['Nota'].max().reset_index().groupby(['Codigo Alumno'])['Nota'].mean().reset_index()

#Seteo el nombre a las columnas para que no se me superpongan
finales_max_df.columns=['Codigo Alumno','Nota_max_prom']

#Genero el resto de las agrupaciones sobre el set de finales
finales_df=finales.groupby(['Codigo Alumno']).agg({'noAprobado':[sum],'Aprobado':[sum],'Promociono':[sum],'Nota':['mean']})

#Corrijo los nombres de las columnas
finales_df.columns = [col[0] for col in finales_df.columns.values]

#Le agrego a los finales la columna de finales maximos agrupados
finales_df=finales_df.reset_index().join(finales_max_df.set_index('Codigo Alumno'),on='Codigo Alumno')

In [ ]:
## Agrego la columna 'Indice_aprobación', la cual se calcula como la cantidad de finales aprobados sobre el total de finales rendidos. 
finales_df['Indice_aprobacion'] = (finales_df['Aprobado'])/(finales_df['Aprobado']+finales_df['noAprobado'])

### **AGRUPACIÓN de todos los registros** ###

#### merged_df

In [ ]:
merged_df = alumnos_df.join(cursadas_df.set_index('Codigo Alumno'),on='Codigo Alumno').join(finales_df.set_index('Codigo Alumno'),on='Codigo Alumno')
merged_df=merged_df[(merged_df['Codigo Alumno'].isin(finales_df['Codigo Alumno'])) & (merged_df['Codigo Alumno'].isin(cursadas_df['Codigo Alumno']))]

In [ ]:
## Generamos la variable 'cantidad de años', la cual indica la cantidad de años transcurridos desde que el alumno se inscribió hasta su última actividad registrada.
merged_df['cantidad de años'] = merged_df['Ciclo Lectivo de Cursada'] - merged_df['Año de ingreso']

In [ ]:
## Nos desprendemos de la variable 'Año de ingreso'.
merged_df.drop(['Año de ingreso'], axis=1, inplace = True)
merged_df.shape

(4558, 29)

In [ ]:
#
merged_df[(merged_df['Codigo Alumno'].isin(finales_df['Codigo Alumno'])) | (merged_df['Codigo Alumno'].isin(cursadas_df['Codigo Alumno']))]

,Codigo Alumno,EsTecnico,deserto,Ciclo Lectivo de Cursada,edad al ingreso,Turno_Mañana,Turno_Noche,Turno_Tarde,Tipo de aprobación_Cambio Curso,Tipo de aprobación_Firmo,...,Descripción de recursada regular_Recurso 4 Veces,Descripción de recursada regular_Recurso 5 Veces,Descripción de recursada regular_Recurso n Veces (>5),noAprobado,Aprobado,Promociono,Nota,Nota_max_prom,Indice_aprobacion,cantidad de años
0,10000007,NaN,1,2014.0,22.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,10.000000,10.000000,1.000000,0.0
2,10000015,NaN,1,2014.0,35.0,0.0,4.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,14.0,0.0,9.928571,9.928571,1.000000,0.0
3,10000016,NaN,1,2014.0,34.0,0.0,6.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,10.000000,10.000000,1.000000,1.0
4,10000020,NaN,1,2014.0,20.0,4.0,3.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,3.0,1.0,9.000000,9.000000,1.000000,0.0
5,10000027,NaN,0,2014.0,29.0,1.0,6.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,3.0,0.0,8.333333,8.333333,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8329,9999960,NaN,0,2014.0,23.0,0.0,6.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,8.0,3.0,9.000000,9.000000,1.000000,0.0
8330,9999966,NaN,0,2014.0,27.0,0.0,8.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,10.000000,10.000000,1.000000,0.0
8331,9999983,NaN,1,2014.0,28.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,7.0,0.0,6.000000,7.142857,0.777778,0.0
8333,9999989,NaN,0,2014.0,25.0,1.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,21.0,2.0,9.761905,9.761905,1.000000,0.0


In [ ]:
#Guardo los registros en un csv
merged_df.to_csv(root_path + 'datos/baseline_2009_0.csv',index=False)

In [ ]:
#
merged_df['deserto'].value_counts()

0    2558
1    2000
Name: deserto, dtype: int64

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4558 entries, 0 to 8334
Data columns (total 29 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Codigo Alumno                                          4558 non-null   int64  
 1   EsTecnico                                              3951 non-null   float64
 2   deserto                                                4558 non-null   int64  
 3   Ciclo Lectivo de Cursada                               4558 non-null   float64
 4   edad al ingreso                                        4558 non-null   float64
 5   Turno_Mañana                                           4558 non-null   float64
 6   Turno_Noche                                            4558 non-null   float64
 7   Turno_Tarde                                            4558 non-null   float64
 8   Tipo de aprobación_Cambio Curso                 